 <blockquote>
 Isaac Callison<br>
 CSCI 6350-001<br>
 Project 4<br>
 Due: 02/22/2020<br>
 </blockquote>

In [112]:
## Import all the things. 
import numpy as np
import csv
import pandas as pd
import random
import nltk
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from nltk.corpus import opinion_lexicon as op
from nltk.tokenize import word_tokenize
from nltk.tokenize import PunktSentenceTokenizer
from nltk import pos_tag
from collections import defaultdict
np.set_printoptions(linewidth=400)   # optional: widens column of numpy array display

In [113]:
## get positve and negative words from the lexicon. 
pos_words = set(op.words('positive-words.txt'))
neg_words = set(op.words('negative-words.txt'))

In [114]:
## create initial list to work on 
X = [] ; y = []; initial_set = []; X_test = []; y_test = []
with open("reviews.txt", 'r', encoding='utf8') as reviews:
    for review in reviews:
        if review != "":
            review = review.strip()
            review = review.lower()
            #review = word_tokenize(review)
            if len(review) > 1:
                initial_set.append(review)
num_sentences = len(initial_set)

In [115]:
########## Since the data predominated with 5's I developed a cumbersome method to makes sure the data trained on was 
############# representative of the different classes. I made sure the poorest represented class was fully represented.
my_dict = defaultdict(list)

### Append each review to a rating key
for item in (initial_set):
    my_dict[int(item[-1])].append(item)

### Find the shortest list of reviews, in this case it is 1    
leng = 1000000
for key,value in my_dict.items():
    v = -1
    if len(value) < leng:
        leng = len(value)
        v = key

#### write out each rating to the training file making sure each is represented        
count = 0
with open("reviews-train.txt", 'w') as r_train:
    for idx in range(0,leng):
        for key,value in my_dict.items():
             r_train.write(value[idx])
             r_train.write('\n')   
             count+=1

#### write the rest out to a file.                 
test_set = initial_set[count:]
with open("reviews-test.txt", 'w') as r_test:
    for line in test_set:
        r_test.write(line)
        r_test.write('\n')

In [116]:
########## this was my attempt to develop a rating system based on number of positives versus number of negatives.

def return_rating(posCount, negCount):
        stars = 0
        if negCount > (posCount*2):
            stars = 1
        elif posCount > (negCount*2):
            stars = 5
        elif negCount == posCount:
            stars = 3
        elif negCount < posCount:
            stars = 4
        elif posCount < negCount:
            stars = 2
        return stars
            

In [117]:
######## open each file, read into memory, strip excess white space, and break into target and features. 
rev_train_file = open('reviews-train.txt')
lines = rev_train_file.readlines()
for line in lines:
    line = line.strip()
    X.append(word_tokenize(line[:-1]))
    y.append(line[-1])
 
rev_train_file.close()

In [118]:
######## open each file, read into memory, strip excess white space, and break into target and features. 
rev_test_file = open('reviews-test.txt')
lines = rev_test_file.readlines()
for line in lines:
    line = line.strip()
    X_test.append(word_tokenize(line[:-1]))
    y_test.append(line[-1])
rev_test_file.close()

In [119]:
### This function just returned the number of positive and negative words from the opinion lexicon

def pos_neg_count(line):
    posCount = 0; negCount = 0
    for word in line:
        if word in pos_words:
            posCount +=1
        elif word in neg_words:
            negCount+=1
    return posCount, negCount
    

In [120]:
########## These are the most common tags across the review set ######
def superlative_count(tokenized, pos_d):
    pos_dict = pos_d
    for i in tokenized[:1]:
        words = nltk.word_tokenize(i)
        tagged = nltk.pos_tag(words)
    ######## Array is incremented if one of these parts of speech shows up. 
     ########0,1,2,3,4,5,6,7,8,9,10
    count = [0,0,0,0,0,0,0,0,0,0,0]
    for x in tagged:
        POS = x[1]
        pos_dict[POS] = pos_dict.get(POS,0) +1
        if(POS) == 'RB':
            count[0] +=1
        if(POS) == 'NN':
            count[1] +=1
        if(POS) == 'VBP':
            count[2] +=1
        if(POS) == 'JJS':
            count[3] +=1
        if(POS) == 'IN':
            count[4] +=1
        if(POS) == 'DT':
            count[5] +=1
        if(POS) == 'NNS':
            count[6] +=1
        if(POS) == 'VBZ':
            count[7] +=1   
        if(POS) == 'JJ':
            count[8] +=1
        if(POS) == 'EX':
            count[9] +=1     
    ######## The array of counts and the accumulating POS dictionary are returned. 
    return count, pos_dict


In [121]:
###################################################################################################################
############################################ FEATURE EXTRACTION ###################################################
###################################################################################################################
def np_feature_extraction(X,y, pos_dict):
    X_list = [] ; y_list = []
    pos_d = pos_dict
    punktLine = PunktSentenceTokenizer()
    for x,line in enumerate(X):
        ### GET TARGET #####
        rating = int(y[x])
     
        # GET POS AND NEG COUNTS
        posCount, negCount = pos_neg_count(line)
        
        ## POS TAGGING
        sent = ' '.join(line)
        token = punktLine.tokenize(sent)
        superlative,pos = superlative_count(token, pos_d)
        
        ### Apply guessed rating
        rating_result = return_rating(posCount, negCount)
        
        ## THIS IS THE NUMPY LINE WITH FEATURES
        temp_line = [posCount, negCount, rating_result] + superlative
        line = temp_line
        #print(line)
     
        ### APPEND LINES #####
        X_list.append(line)
        y_list.append(rating)
        
    ##TURN INTO NUMPY ARRAYS
    X_np = (np.array(X_list))
    y_np = (np.array(y_list))
    
    return X_np,y_np, pos_d
#########################################################################################################################

In [122]:
pos_dict = {}
X_train_np, y_train_np, pos = np_feature_extraction(X,y, pos_dict)
X_test_np, y_test_np, pos = np_feature_extraction(X_test, y_test, pos_dict)

############ This is the total number of each of the speech tags in the document. 
#### Uncomment to see the dictionary:
#print(pos)

In [123]:
for mclass in ('multinomial', 'ovr'):
    lr = LogisticRegression(solver='lbfgs', max_iter=3000, random_state=0, multi_class=mclass).fit(X_train_np, y_train_np)
    yhat = lr.predict(X_test_np)
     
    # the 3 lines below show how to invoke various output    
    print("\n",mclass," Accuracy",accuracy_score(y_test_np, yhat))
    print("\n",mclass," Classification Report\n",classification_report(y_test_np, yhat),sep="")
    print("\n",mclass," Confusion Matrix\n",confusion_matrix(y_test_np, yhat),sep="")


 multinomial  Accuracy 0.3841707425522454

multinomial Classification Report
              precision    recall  f1-score   support

           1       0.09      0.51      0.16        43
           2       0.04      0.15      0.07        61
           3       0.10      0.25      0.14       161
           4       0.24      0.24      0.24       464
           5       0.75      0.45      0.56      1520

    accuracy                           0.38      2249
   macro avg       0.24      0.32      0.23      2249
weighted avg       0.57      0.38      0.44      2249


multinomial Confusion Matrix
[[ 22   4   8   4   5]
 [ 17   9  13  10  12]
 [ 36  17  41  29  38]
 [ 46  54  84 113 167]
 [111 130 278 322 679]]

 ovr  Accuracy 0.3863939528679413

ovr Classification Report
              precision    recall  f1-score   support

           1       0.09      0.56      0.15        43
           2       0.03      0.08      0.05        61
           3       0.10      0.24      0.14       161
        

<blockquote>

Selection of features – Explain how you arrived at your final set of features: what did you try? What insight lead you to them? Were there one or more that were particularly good?

ANSWER: I tried length of review, but that gave me worse results. I tried one hot encoding, but that did not seem to give any better results than just a basic rating system based on positive and negative words from the NLTK lexicon. I included the postive and negative words from the lexicon as raw numbers. I tried POS tagging by applying points based on the type of superlative. Then I created a dictionary to count the number of times each part of speech actually showed up, counted them as features, and inserted them into my feature list. 
    
With regard to the training and testing data, I found that the program could not discern any other categories except for 5 star ratings, and that this was probably due to the fact that 5 star ratings predominated in the reviews.txt file. Unfortunately I realized this late, and my implementation of a balanced training set resulted in a 10/90 split of training/testing data.   

Provide your thoughts on the differences observed between one versus rest and multinomial classification. 

ANSWER: From my scores, there was not a whole lot of difference. I know the OVR method looks at one and differentiates the rest, and the MN looks at differences among all of them. My results were so low I am not sure any true differences came out. 

Provide an analysis of your measures (accuracy, f1, precision, recall) and the confusion matrix. 

ANSWER: My accuracy was very poor regardless of using multinomial or OVR. The multinomial method did show slightly higher results over the OVR. Recall was highest among the 1 start ratings, but I included every 1 star rating available across the set. Precision and F1 was the highest for the 5 star rating, no surprise as the bulk of the reviews were in fact 5 stars. 
    
</blockquote>
